In [203]:
#DOMAIN DRIVEN DESIGN:
#Problem: create an application that has users, each of whom has a balance that can update.

#The application will have 3 parts: Database, Authentication service, UI

class Domain:
    def __init__(self, name):
        self.name = name

    def do_something(self, *args, **kwargs):
        pass

    def __repr__(self):
        return f"<Domain: {self.name}>"


In [204]:
#Lets add Database Domain
#each Domain has Aggregates : for us its self.databases
#each aggregate has entities


class Database:
    def __init__(self, name):
        self.name = name

class DataDomain(Domain):
    def __init__(self, name, databases:  [Database]):
        super().__init__(name)
        self.databases = dict([(db.name, db) for db in databases])

    def get_database(self, name):
        return self.databases.get(name, None)


In [205]:
#we need a user database, what should it have?


#1.

#2.

#3.

#4.







In [206]:

from dataclasses import dataclass
@dataclass
class User:
    id: int
    username: str
    password: str = '0000'
    balance: int = 0

#CRUD: we can check if user exists, add user, update user, delete.

#lets define an interface
class Userbase(Database):
    def __init__(self, name):
        super().__init__(name)

    def get_user(self, id, password):
        return User(id, 'user')

    def add_user(self, user: User):
        pass #or notimplementexception

    def update_user(self, user: User):
        pass

    def delete_user(self, id):
        pass


In [207]:
#Add a simple implementation

class SimpleUserbase(Userbase):
    def __init__(self, name, users:dict[int, User] = None):
        super().__init__(name)
        self.users = users if users else {1: User(1, 'user')}

    def get_user(self, username):
        for user in self.users.values():
            if user.username == username:
                return user
        return None

    def add_user(self, user: User):
        if user.id not in self.users:
            self.users[user.id] = user
            return True
        return False

    def update_user(self, user: User):
        if user.id not in self.users:
            return False
        self.users[user.id] = user
        return True

    def delete_user(self, id):
        if id in self.users:
            del self.users[id]
            return True
        return False



In [208]:
user_db = SimpleUserbase(name = 'user_db')
data_domain = DataDomain(name = 'data_domain', databases = [user_db])


In [209]:
#how do we reach the database?

class Authentication:
    def __init__(self, database: Userbase):
        self.database = database

    def check_user(self, username):
        return self.database.get_user(username) is not None

    def authenticate(self, username, password):
        user = self.database.get_user(username)
        if user and user.password == password:
            return user
        return None

    def store_user(self, user: User):
        return self.database.add_user(user)


class AuthenticationDomain(Domain):
    def __init__(self, name, database: Userbase):
        super().__init__(name)
        self.database = database

    def get_user_authentication(self):
        return Authentication(self.database)


In [210]:
#Front-end
class UI(Domain):
    def __init__(self, name, authentication: Authentication):
        super().__init__(name)
        self.authentication = authentication

    def get_login_vm(self, id, password):
        pass



In [211]:
class ViewModel:
    def __init__(self, name: str):
        self.name = name

    def display(self):
        #list api of vm
        pass

    def __repr__(self):
        print(f"To see available functionality, call .display()")


class LoginViewModel(ViewModel):
    def __init__(self, authentication: Authentication):
        super().__init__('login_vm')
        self.authentication = authentication

    def display(self):
        print(f".login(username, password)\n.validate(username")

    def validate(self, username):
        return self.authentication.check_user(username)

    def login(self, username, password):
        user = self.authentication.authenticate(username, password)
        if user:
            print(f"Welcome {user.username}")
            return None #interaction vm
        else:
            print("Invalid username or password")
            return None




In [212]:
class InteractionViewModel(ViewModel):
    def __init__(self, user: User):
        super().__init__('interaction_vm')
        self.user = user

    def display(self):
        print(f".update_balance(amount)\n.show_balance()")
        print(f"when done, call ui.save(this_vm)")

    def update_balance(self, amount):
        self.user.balance += amount

    def show_balance(self):
        print(f"Balance: {self.user.balance}")



In [213]:
#LETS REDO APPROPRIATELY

In [214]:


class LoginViewModel(ViewModel):
    def __init__(self, authentication: Authentication):
        super().__init__('login_vm')
        self.authentication = authentication

    def display(self):
        print(f".login(username, password)\n.validate(username")

    def validate(self, username):
        return self.authentication.check_user(username)

    def login(self, username, password):
        user = self.authentication.authenticate(username, password)
        if user:
            print(f"Welcome {user.username}")
            return InteractionViewModel(user)
        else:
            print("Invalid username or password")
            return None

class ValidateViewModel(ViewModel):
    def __init__(self, authentication: Authentication):
        super().__init__('validate_vm')
        self.authentication = authentication

    def display(self):
        print(f".validate(username)")

    def validate(self, username):
        validation = self.authentication.check_user(username)
        if validation:
            print(f"Username {username} is valid")
        else:
            print(f"Username {username} is invalid")
        return validation

#Front-end
class UI(Domain):
    def __init__(self, name, authentication: Authentication):
        super().__init__(name)
        self.authentication = authentication

    def get_login_vm(self):
        return LoginViewModel(self.authentication)

    def get_validate_vm(self, username):
        return ValidateViewModel(self.authentication)

    def save(self, vm):
        self.authentication.store_user(vm.user)


In [215]:
#Now we're ready to build the application!

In [217]:
def build_application():
    user_db = SimpleUserbase(name = 'user_db')
    data_domain = DataDomain(name = 'data_domain', databases = [user_db])
    authentication = AuthenticationDomain(name = 'authentication', database = data_domain.get_database('user_db'))
    ui = UI(name = 'ui', authentication = authentication.get_user_authentication())
    return ui

ui = build_application()

In [218]:
#Lets use our application!
validate_vm = ui.get_validate_vm('user')
validate_vm.display()
validate_vm.validate('user')

.validate(username)
Username user is valid


True

In [219]:
login_vm = ui.get_login_vm()
login_vm.display()
interactive_vm = login_vm.login('user', '0000')
interactive_vm.display()
interactive_vm.update_balance(100)
interactive_vm.show_balance()
ui.save(interactive_vm)

.login(username, password)
.validate(username
Welcome user
.update_balance(amount)
.show_balance()
when done, call ui.save(this_vm)
Balance: 100


In [220]:

ui.authentication.database.users

{1: User(id=1, username='user', password='0000', balance=100)}

In [200]:
#DESIGN PATTERNS!

#https://refactoring.guru/design-patterns

In [221]:
#We need system to directly inject the database into authentication_domain - DEPENDENCY
#we need system to directly inject user authentication into ui domain - DEPENDENCY

#PATTERN ONE: Registrar/Resolver

class Registrar:
    def __init__(self):
        pass

    def register(self, entity_name: str, entity: object):
        pass

class Resolver:
    def __init__(self):
        pass

    def resolve(self, entity_name: str):
        pass

class RegistrarResolver(Registrar, Resolver):
    def __init__(self):
        super().__init__()
        self.inventory = {}

    def register(self, entity_name: str, entity: object):
        self.inventory[entity_name] = entity

    def resolve(self, entity_name: str):
        return self.inventory.get(entity_name, None)








In [ ]:
#Singletons / services
#Sinleton benefits:
#1. Thread safety
#2. no need for duplicate memory
#3.other


#We want to add a logger!
#our logger will be 'thread safe' and only 'write-to-disk' on flush

#We want to add a custom output screen!
#service - stateless

In [ ]:
class Logger:
    def __init__(self):
        self.buffer = []
        self.lock = False

    def log(self, message):
        def sleep(seconds):
            pass
        while self.lock:
            sleep(1)

        #acquire lock
        self.lock = True
        #perform operation
        self.buffer.append(message)
        #release lock
        self.lock = False

    def flush(self):
        #write out to memory
        self.buffer = []
        pass

In [222]:
class OutputScreen:
    def __init__(self, add_exclamation: bool = False):
        self.add_exclamation = add_exclamation
        pass

    def show(self, message):
        print(message)
        if self.add_exclamation:
            print('!')

In [225]:
#OBSERVER - SUBSCRIBER

#OBSERVER SUBSCRIBER is a kind of MQ system that allows us to have a single source of truth

#1. Say you have multiple view models / components / frames that use information about an object (maybe stock)
#That information needs to be periodically updated based on a url GET
#Instead of manually updating every single dependant view with a separate GET
#we can have a single source of truth that decides to update and then the interested components who listen to it (subscribers) get updated automagically
#The views can also be thought of as separate devices. If an update occurs in a database (maybe a news update), now all devices that subscribe to this newsletter will get the newest news
#This is referred to as a PUSH model as opposed to a PULL model
#Think of it as push notifications vs mail that only updates when you open the inbox

#Rx is a great library for this1